Célula 1 — Configuração inicial
Carregamento das bibliotecas Python

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
# Redefine regression_metrics to fix the 'squared' argument issue
from xgboost import XGBRegressor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

Célula 2 — Definição das colunas

In [2]:
# Electrical and harmonic features (as in Sensors-25-04601)
ELECTRICAL_COLS = [
    "irms", "vrms", "power_factor",
    "p_apparente", "p_active"
]

HARMONIC_COLS = [f"h{i}" for i in range(1, 33)]

FEATURE_COLS = ELECTRICAL_COLS + HARMONIC_COLS

Célula 3 — Loader oficial do dataset

In [3]:
def load_nilm_dataset(data_root):
    """
    Loads the NILM dataset organized by acquisition sessions and sensor-phase files.

    Dataset structure:
    data/
      ├── session_1/
      │    ├── S1P1.csv
      │    ├── ...
      ├── session_2/
           ├── ...

    Returns
    -------
    pd.DataFrame
        Unified dataframe with metadata columns.
    """
    data_root = Path(data_root)
    dfs = []

    for session_dir in sorted(data_root.iterdir()):
        if not session_dir.is_dir():
            continue

        session_name = session_dir.name

        for csv_file in sorted(session_dir.glob("*.csv")):
            filename = csv_file.stem  # e.g., S1P1
            sensor = filename.split("P")[0]      # S1
            phase = "P" + filename.split("P")[1] # P1

            df = pd.read_csv(csv_file)

            # Keep only relevant columns
            df = df[["time"] + FEATURE_COLS]

            # Metadata (important for analysis)
            df["session"] = session_name
            df["sensor"] = sensor
            df["phase"] = phase
            df["channel"] = f"{sensor}_{phase}"

            dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

Célula 4 — Carregamento do dataset no Colab

In [4]:
# If using Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/nilm-dataset-main/nilm-dataset-main/data"  # ajuste se necessário

df = load_nilm_dataset(DATA_PATH)

print("Dataset loaded successfully!")
print("Shape:", df.shape)
df.head()


Dataset loaded successfully!
Shape: (1612718, 42)


,time,irms,vrms,power_factor,p_apparente,p_active,h1,h2,h3,h4,...,h27,h28,h29,h30,h31,h32,session,sensor,phase,channel
0,124,0.0,0.00,0.00,0,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,05-12 8h,S1,P1,S1_P1
1,134,0.0,0.00,0.00,0,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,05-12 8h,S1,P1,S1_P1
2,144,0.0,0.00,0.00,0,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,05-12 8h,S1,P1,S1_P1
3,146,0.0,224.90,0.21,1,0,0.000816,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,05-12 8h,S1,P1,S1_P1
4,148,0.0,225.84,0.26,1,0,0.000816,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,05-12 8h,S1,P1,S1_P1


Célula 5 — Validação

In [5]:
assert all(col in df.columns for col in FEATURE_COLS), "Missing feature columns"
assert df.isnull().sum().sum() == 0, "Dataset contains missing values"

df.describe()

,time,irms,vrms,power_factor,p_apparente,p_active,h1,h2,h3,h4,...,h23,h24,h25,h26,h27,h28,h29,h30,h31,h32
count,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,...,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06,1.612718e+06
mean,1.285849e+04,2.986589e-01,2.391017e+02,3.800290e-01,7.115135e+01,5.895274e+01,1.282875e-01,4.102651e-03,3.699963e-02,1.654518e-03,...,3.116220e-03,3.192590e-04,2.793487e-03,2.743226e-04,2.315361e-03,2.733475e-04,1.800771e-03,2.670634e-04,1.627056e-03,2.722632e-04
std,7.746003e+03,9.129399e-01,4.105178e+00,3.549899e-01,2.134926e+02,2.109770e+02,4.519433e-01,2.587412e-02,5.607231e-02,8.418493e-03,...,4.776471e-03,1.035078e-03,4.339745e-03,7.642839e-04,3.710752e-03,7.629300e-04,2.936258e-03,7.873431e-04,2.513844e-03,6.495557e-04
min,2.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.256000e+03,0.000000e+00,2.380500e+02,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.249200e+04,4.000000e-02,2.395500e+02,4.900000e-01,8.000000e+00,4.000000e+00,1.061300e-02,0.000000e+00,8.804000e-03,0.000000e+00,...,9.980000e-04,0.000000e+00,9.160000e-04,0.000000e+00,6.980000e-04,0.000000e+00,4.080000e-04,0.000000e+00,5.800000e-04,0.000000e+00
75%,1.893500e+04,1.900000e-01,2.408100e+02,6.700000e-01,4.500000e+01,4.000000e+01,8.898800e-02,7.110000e-04,2.824200e-02,7.040000e-04,...,3.044000e-03,4.080000e-04,2.296000e-03,4.080000e-04,1.534000e-03,4.080000e-04,1.316000e-03,4.070000e-04,1.415000e-03,4.090000e-04
max,2.892400e+04,1.515000e+01,2.459300e+02,1.000000e+00,3.239000e+03,3.225000e+03,7.847237e+00,2.252210e-01,4.086770e-01,9.727600e-02,...,4.608900e-02,4.521500e-02,4.359300e-02,4.253800e-02,4.101800e-02,4.045900e-02,3.919200e-02,3.898700e-02,3.791600e-02,3.735700e-02


Célula 6 — Preparação para treino

In [6]:
X = df[FEATURE_COLS].values
y = df["p_active"].values  # or appliance label if available


Célula 7 — Split por sessão

In [7]:
from sklearn.model_selection import train_test_split

train_sessions = df["session"].unique()[:int(0.7 * df["session"].nunique())]

train_idx = df["session"].isin(train_sessions)
test_idx  = ~train_idx

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

Célula 8 — Normalização e scaling

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

Célula 9 — Métricas padrão NILM

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np # Import numpy for sqrt

def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred) # Calculate MSE
    rmse = np.sqrt(mse) # Take square root for RMSE
    nrmse = rmse / (y_true.max() - y_true.min())
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, nrmse, r2

1. Linear Regression (baseline)

In [26]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
mae_lr, rmse_lr, nrmse_lr, r2_lr = regression_metrics(y_test, y_pred_lr)

In [27]:
mae_lr, rmse_lr, nrmse_lr, r2_lr


(5.454709424407734e-14,
 np.float64(1.1574391255632856e-13),
 np.float64(4.006365959028334e-17),
 1.0)

2. KNN Regressor

In [22]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5, weights="distance")
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
mae_knn, rmse_knn, nrmse_knn, r2_knn = regression_metrics(y_test, y_pred_knn)

In [23]:
mae_knn, rmse_knn, nrmse_knn, r2_knn

(0.042942699740298705,
 np.float64(0.40833828482527174),
 np.float64(0.00014134243157676418),
 0.9999961152955421)

3. Decision Tree Regressor

In [20]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=10, random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
mae_dt, rmse_dt, nrmse_dt, r2_dt = regression_metrics(y_test, y_pred_dt)


In [21]:
mae_dt, rmse_dt, nrmse_dt, r2_dt

(0.05800850500350839,
 np.float64(1.0204319626195448),
 np.float64(0.00035321286348893903),
 0.9999757402740657)

4. Random Forest Regressor

In [18]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mae_rf, rmse_rf, nrmse_rf, r2_rf = regression_metrics(y_test, y_pred_rf)


In [19]:
mae_rf, rmse_rf, nrmse_rf, r2_rf

(0.013933373837777886,
 np.float64(0.5539807323516417),
 np.float64(0.0001917551859991837),
 0.9999928499825188)

5. Gradient Boosting Regressor

In [16]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, random_state=42)
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)
mae_gbr, rmse_gbr, nrmse_gbr, r2_gbr = regression_metrics(y_test, y_pred_gbr)


In [17]:
mae_gbr, rmse_gbr, nrmse_gbr, r2_gbr

(0.049121052234671184,
 np.float64(0.33413370367223505),
 np.float64(0.00011565721830122362),
 0.9999973988921923)

6. XGBoost Regressor

In [14]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    objective="reg:squarederror",
    random_state=42
)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
mae_xgb, rmse_xgb, nrmse_xgb, r2_xgb = regression_metrics(y_test, y_pred_xgb)


In [15]:
mae_xgb, rmse_xgb, nrmse_xgb, r2_xgb

(0.5797379016876221,
 np.float64(9.633087723536804),
 np.float64(0.0033344021196042932),
 0.997838020324707)

7. LightGBM Regressor

In [12]:
import lightgbm as lgb

lgbm = lgb.LGBMRegressor(
    n_estimators=200,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)
mae_lgbm, rmse_lgbm, nrmse_lgbm, r2_lgbm = regression_metrics(y_test, y_pred_lgbm)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.282681 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9246
[LightGBM] [Info] Number of data points in the train set: 1064630, number of used features: 37
[LightGBM] [Info] Start training from score 58.951282


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [13]:
mae_lgbm, rmse_lgbm, nrmse_lgbm, r2_lgbm

(0.17686805104438194,
 np.float64(1.8541851573968005),
 np.float64(0.0006418086387666323),
 0.9999199015736179)

8. MLP Regressor

In [10]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(
    hidden_layer_sizes=(128, 64),
    activation="relu",
    max_iter=300,
    random_state=42
)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)
mae_mlp, rmse_mlp, nrmse_mlp, r2_mlp = regression_metrics(y_test, y_pred_mlp)


In [11]:
mae_mlp, rmse_mlp, nrmse_mlp, r2_mlp

(0.02974392013132596,
 np.float64(0.06796671683879636),
 np.float64(2.3526035596675792e-05),
 0.9999998923756933)

9. 2seqpoint

In [ ]:
def create_seq2point_windows(X, y, window_size=99):
    X_seq, y_seq = [], []
    half = window_size // 2

    for i in range(half, len(X) - half):
        X_seq.append(X[i-half:i+half+1])
        y_seq.append(y[i])

    return np.array(X_seq), np.array(y_seq)

WINDOW_SIZE = 49
BATCH_SIZE = 64
X = X.astype(np.float32)
y = y.astype(np.float32)


X_train_seq, y_train_seq = create_seq2point_windows(X_train_scaled, y_train, WINDOW_SIZE)
X_test_seq,  y_test_seq  = create_seq2point_windows(X_test_scaled,  y_test,  WINDOW_SIZE)



class Seq2Point(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.conv1 = nn.Conv1d(n_features, 30, kernel_size=10)
        self.conv2 = nn.Conv1d(30, 30, kernel_size=8)
        #self.fc1 = nn.Linear(30 * 82, 1024)
        self.fc1 = nn.Linear(990, 1024)

        self.fc2 = nn.Linear(1024, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        return self.fc2(x).squeeze()

model = Seq2Point(X_train_seq.shape[2])

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

train_loader = DataLoader(
    TensorDataset(
        torch.tensor(X_train_seq, dtype=torch.float32),
        torch.tensor(y_train_seq, dtype=torch.float32)
    ),
    batch_size=64,
    shuffle=True
)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()

model.eval()
with torch.no_grad():
    y_pred_seq = model(
        torch.tensor(X_test_seq, dtype=torch.float32).to(device)
    ).cpu().numpy()

mae_seq, rmse_seq, nrmse_seq, r2_seq = regression_metrics(y_test_seq, y_pred_seq)

Tabela Final

In [28]:
import pandas as pd

results = pd.DataFrame({
    "MAE": {
        "Linear Regression": mae_lr,
        "KNN": mae_knn,
        "Decision Tree": mae_dt,
        "Random Forest": mae_rf,
        "Gradient Boosting": mae_gbr,
        "XGBoost": mae_xgb,
        "LightGBM": mae_lgbm,
        "MLP": mae_mlp

    },
    "RMSE": {
        "Linear Regression": rmse_lr,
        "KNN": rmse_knn,
        "Decision Tree": rmse_dt,
        "Random Forest": rmse_rf,
        "Gradient Boosting": rmse_gbr,
        "XGBoost": rmse_xgb,
        "LightGBM": rmse_lgbm,
        "MLP": rmse_mlp
    },
    "NRMSE": {
        "Linear Regression": nrmse_lr,
        "KNN": nrmse_knn,
        "Decision Tree": nrmse_dt,
        "Random Forest": nrmse_rf,
        "Gradient Boosting": nrmse_gbr,
        "XGBoost": nrmse_xgb,
        "LightGBM": nrmse_lgbm,
        "MLP": nrmse_mlp

    },
    "R2": {
        "Linear Regression": r2_lr,
        "KNN": r2_knn,
        "Decision Tree": r2_dt,
        "Random Forest": r2_rf,
        "Gradient Boosting": r2_gbr,
        "XGBoost": r2_xgb,
        "LightGBM": r2_lgbm,
        "MLP": r2_mlp

    }
})

results


,MAE,RMSE,NRMSE,R2
Linear Regression,5.454709e-14,1.157439e-13,4.006366e-17,1.000000
KNN,4.294270e-02,4.083383e-01,1.413424e-04,0.999996
Decision Tree,5.800851e-02,1.020432e+00,3.532129e-04,0.999976
Random Forest,1.393337e-02,5.539807e-01,1.917552e-04,0.999993
Gradient Boosting,4.912105e-02,3.341337e-01,1.156572e-04,0.999997
XGBoost,5.797379e-01,9.633088e+00,3.334402e-03,0.997838
LightGBM,1.768681e-01,1.854185e+00,6.418086e-04,0.999920
MLP,2.974392e-02,6.796672e-02,2.352604e-05,1.000000
